### The setup

In [36]:
import requests
import json 
import pandas as pd
from time import sleep

In [37]:
%pwd

'/Users/hayatoumy/Documents/GA files/git_lessons/project_4'

In [38]:
with open('./consumer_key.txt') as f1:
    consumer_key = f1.read().strip()

with open('./consumer_secret.txt') as f2:
    consumer_secret = f2.read().strip()

with open('./access_token.txt') as f3:
    access_token = f3.read().strip()

with open('./access_token_secret.txt') as f4:
    access_token_secret = f4.read().strip()

In [39]:
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

### The cities name and Twitter usernames data frames

In [4]:
from bs4 import BeautifulSoup

Gathering the most flooded cities in the US from the link below. <br /> 
Gathering the traffic Twitter usernames for each of these cities manually. <br />
Putting them together in a data frame for quick retreival. 

In [5]:
flood_url = 'https://www.cheatsheet.com/culture/american-cities-homes-danger-flooding.html/'

In [6]:
flood_content = requests.get(flood_url)

In [7]:
flood_soup = BeautifulSoup(flood_content.content, 'lxml')

In [14]:
cities = []
for h in flood_soup.find_all('h2'):
#     print(h.text.split('.')[1:])
#     print("".join(h.text.split('.')[1:]))
    cities.append("".join(h.text.split('.')[1:]))

In [15]:
# cleaning up the city names and location:
cities.index(' St Augustine')

8

In [16]:
# adding the state to St Augustine
cities[8] = " St Augustine, Florida"

In [17]:
cities = [i.strip() for i in cities]
cities

['Pahrump, Nevada',
 'Jersey City, New Jersey',
 'Plano, Texas',
 'Peachtree Corners, Georgia',
 'Miramar Beach, Florida',
 'Raleigh, North Carolina',
 'Rogers City, Arkansas',
 'Carmel, Indiana',
 'St Augustine, Florida',
 'The Woodlands, Texas',
 'Round Rock, Texas',
 'Denton, Texas',
 'Kissimmee, Florida',
 'San Marcos, Texas',
 'Sugar Land, Texas']

In [18]:
twitter_user_names = [
    ["nevadadot"],
    ["NJDOT_info", "511nyNJ"],
    ["TotalTrafficDFW", "krldtraffic", "NTTATravelAlert", "cityofplanotx"],
    ["PtreeCorners"],
    ["fl511_panhandl"],
    ["TotalTrafficRDU", "NCDOT_Triangle", "NCDOT", "RW911", "NCDOT_I77"],
    ["myARDOT", "traffic_nwa", "myARDOT"],
    ["QCTrafficAlerts", "WIBCTraffic"],
    ["fl511_northeast", "SJSOPIO"],
    ["ih45n_traffic", "KPRC2Traffic", "TownshipTransit"],
    ["roundrock"],
    ["DentonTweets", "ScannerRadioDFW", "DFWscanner"],
    ["fl511_central", "MyNews13Traffic", "fl511_state"],
    ["TxDOT", "TotalTrafficAUS"],
    ["SugarLandtxgov", "houstontranstar", "TotalTrafficHOU"]
    
]

In [19]:
data_f = pd.DataFrame(twitter_user_names).T

In [22]:
data_f.columns = cities
data_f

,"Pahrump, Nevada","Jersey City, New Jersey","Plano, Texas","Peachtree Corners, Georgia","Miramar Beach, Florida","Raleigh, North Carolina","Rogers City, Arkansas","Carmel, Indiana","St Augustine, Florida","The Woodlands, Texas","Round Rock, Texas","Denton, Texas","Kissimmee, Florida","San Marcos, Texas","Sugar Land, Texas"
0,nevadadot,NJDOT_info,TotalTrafficDFW,PtreeCorners,fl511_panhandl,TotalTrafficRDU,myARDOT,QCTrafficAlerts,fl511_northeast,ih45n_traffic,roundrock,DentonTweets,fl511_central,TxDOT,SugarLandtxgov
1,None,511nyNJ,krldtraffic,None,None,NCDOT_Triangle,traffic_nwa,WIBCTraffic,SJSOPIO,KPRC2Traffic,None,ScannerRadioDFW,MyNews13Traffic,TotalTrafficAUS,houstontranstar
2,None,None,NTTATravelAlert,None,None,NCDOT,myARDOT,None,None,TownshipTransit,None,DFWscanner,fl511_state,None,TotalTrafficHOU
3,None,None,cityofplanotx,None,None,RW911,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,NCDOT_I77,None,None,None,None,None,None,None,None,None


In [26]:
def get_twitter_user_names(city_name, df = data_f):
    """
    Returns the Twitter user names that you can plug later in the function: traffic, that get twitter updates.
    
    Parameters:
    ----------
    city_name: Enter the city name, state. All small letters, in this format. Example: "round rock, texas".
    df: the dataframe of city names and their traffic Twitter user names. 
    """
    temp_df = data_f
    temp_df.columns = [col.lower() for col in temp_df.columns]
    try: 
        mask = temp_df[city_name].notna()
        return list(temp_df.loc[mask, city_name])
    except KeyError: 
        print(f'This City Does Not Have Twitter User Names Yet. Try From This List: {temp_df.columns}')

In [27]:
# testing the function:
get_twitter_user_names('austin, texas')

This City Does Not Have Twitter User Names Yet. Try From This List: Index(['pahrump, nevada', 'jersey city, new jersey', 'plano, texas',
       'peachtree corners, georgia', 'miramar beach, florida',
       'raleigh, north carolina', 'rogers city, arkansas', 'carmel, indiana',
       'st augustine, florida', 'the woodlands, texas', 'round rock, texas',
       'denton, texas', 'kissimmee, florida', 'san marcos, texas',
       'sugar land, texas'],
      dtype='object')


In [30]:
get_twitter_user_names('plano, texas')

['TotalTrafficDFW', 'krldtraffic', 'NTTATravelAlert', 'cityofplanotx']

Next, making a function that will go collect the most recent traffic tweets for us

In [33]:
def gather_tweets(handle:str, n=300):
    """
    Returns the most recent tweets (up to 200), from the selected Twitter username (handle)
    
    Parameters:
    -----------
    handle: name of the Twitter user page
    n: how many tweets you want. can't get more than 200 
    retrieves only recent tweets
    """
    tweets_everything = api.user_timeline(handle, count = n)
    df = pd.DataFrame(columns = ['id', 'tweets', 'date', 'location'])
    
    for i in tweets_everything:
        tweets = i.text
        try: 
            date = i.formatted_date
        except: 
            date = i.created_at
        
        try:
            location = i.geo['coordinates']
        except: 
            try: 
                location = i.coordinates
            except: 
                location = 'NaN'
                
        tweet_id = i.id # by the way, tweet_id is included in the permalink
                
        df.loc[len(df)] = [tweet_id, tweets, date, location] # inside the loop, building the df row by row
        
    return df

The next function is user-interactive:

In [34]:
def traffic(users = ["TotalTrafficHOU", "houstontranstar"]):
    """
    This function takes an input a street name, and return the most live recent tweets from the traffic data frame.
    Enter street name in all any case you like, upper, lower, mix..
    
    parameters: 
    users: a list of actual Twitter user names (found after the @ in their homepage on Twitter)
    
    """
    print('WARNING: If no users were passed, it will search Houston, TX traffic by default')
    
    if users == []:
        users = ["TotalTrafficHOU", "houstontranstar"]
        print("Empty users list passed; changed to default: Houston, TX traffic")
        
    s = input('What street you want to find? ')
    
    # getting the df, in case there's more than 2 users: 
    lst = []
    for u in users: 
        lst.append(gather_tweets(u))
        
    df = pd.concat(lst, 
              axis = 0, sort = False)
    
    mask = df['tweets'].str.contains(s, case = False) # case = False makes it not case sensitive!

    
    return df.loc[mask, 'tweets'].values[:15]
# the tweets are ordered most recent first by nature; the first 15 tweets are the most recent. 
# since the resutls are not perfect, one will have information about other streets too

In [40]:
# testing the function:
traffic()

What street you want to find? i-10


array(['Accident cleared on I-45 N Fwy Outbound after I-10, stop and go traffic back to Scott, delay of 20 mins #traffic https://t.co/vuUN2yElDh',
       'Accident. three left lanes blocked. on I-45 N Fwy Outbound after I-10, stop and go traffic back to Scott, delay of 20 mins #traffic',
       'Closed due to accident in #Downtown on I-45 N Fwy Outbound after I-10 and before Quitman, stopped traffic back to Scott #traffic',
       'Accident, left lane blocked on I-45 N Fwy Outbound after I-10, stop and go traffic back to Scott, delay of 11 mins #traffic',
       'Accident cleared in #EnergyCorridor on I-10 Katy Fwy Inbound after Kirkwood, stop and go traffic back to Dairy Ashford, delay of 3 mins',
       'Accident, center lane blocked in #EnergyCorridor on I-10 Katy Fwy Inbound after Kirkwood, stop and go traffic back to Dairy Ashford #traffic',
       'Vehicle fire cleared in #Northshore on I-10 E Fwy Outbound after Normandy, stop and go traffic back to Federal, delay of 10 mins #tra

In [46]:
# recall that our options for cities are:
[col for col in data_f]

['pahrump, nevada',
 'jersey city, new jersey',
 'plano, texas',
 'peachtree corners, georgia',
 'miramar beach, florida',
 'raleigh, north carolina',
 'rogers city, arkansas',
 'carmel, indiana',
 'st augustine, florida',
 'the woodlands, texas',
 'round rock, texas',
 'denton, texas',
 'kissimmee, florida',
 'san marcos, texas',
 'sugar land, texas']

In [48]:
get_twitter_user_names('plano, texas')

['TotalTrafficDFW', 'krldtraffic', 'NTTATravelAlert', 'cityofplanotx']

To use this function from a first responder, call the traffic function and pass is the function that get the Twitter usernames, like this:

In [50]:
traffic(get_twitter_user_names('plano, texas'))

What street you want to find? 75


array(['Stalled vehicle. right lane on nb exit ramp blocked in #Dallas on 635 LBJ Fwy WB at 75 Central Expwy #DFWTraffic https://t.co/4NzBppXRvr',
       'Accident cleared in #Dallas on 75 SB approaching NW Hwy, stop and go traffic back to Spg Vly Rd, delay of 16 mins #DFWTraffic',
       'Accident. two left lanes blocked in #Dallas on 75 SB approaching NW Hwy, stop and go traffic back to Spg Vly Rd, delay of 21 mins',
       'Disabled cleared in #Dallas on 75 SB approaching Caruth Haven Ln, stop and go traffic back to Spg Vly Rd, delay of 31 mins #DFWTraffic',
       'Disabled cleared in #Dallas on 75 SB approaching Caruth Haven Ln, stop and go traffic back to 635 Lbj Frwy, delay of 18 mins #DFWTraffic',
       'Disabled vehicle, left lane blocked in #Dallas on 75 SB approaching Caruth Haven Ln, stop and go traffic back to 635 Lbj Frwy #DFWTraffic',
       'Disabled vehicle, left lane blocked in #Dallas on 75 SB approaching Caruth Haven Ln, stop and go traffic back to Royal Ln, delay 